In [ ]:
from pandas import DataFrame, read_csv
import tensorrt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from keras.layers import LSTM, Activation, Dense, Input, Flatten,RepeatVector
from keras.optimizers import RMSprop
from keras import Model,Sequential

# Raw data loading

In [ ]:
raw_df = read_csv('car_data.csv')
print(raw_df.shape)
raw_df.describe()